In [24]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool
from azure.identity import DefaultAzureCredential
from datetime import datetime
import os
from dotenv import load_dotenv

load_dotenv()

True

In [25]:
def create_code_interpreter_agent(client):
    """Creates an AI agent with code execution capabilities."""
    code_interpreter = CodeInterpreterTool()
    agent = client.agents.create_agent(
        model=os.getenv("AZURE_API_DEPLOYMENT"),
        name="code-interpreter-agent",
        instructions="You are a helpful Python programming assistant.",
        tools=code_interpreter.definitions,
        tool_resources=code_interpreter.resources,
    )
    return agent

In [26]:
def create_thread_and_message(client, agent_id: str, user_input: str):
    """Creates a new thread and sends a user message."""
    thread = client.agents.create_thread()
    message = client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=f"""
        Write a blog on the given topic:
        1. Save {user_input} to blog-{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.md
        2. Provide a download link for the saved file.
        """,
    )
    return thread

In [27]:
def execute_run(client, thread_id: str, agent_id: str):
    """Executes the agent's task and handles errors."""
    run = client.agents.create_and_process_run(thread_id=thread_id, assistant_id=agent_id)
    print(f"Run finished with status: {run.status}")
    
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
        return None
    
    return run

In [28]:
def fetch_and_save_file(client, thread_id: str):
    """Fetches the generated file and saves it locally."""
    messages = client.agents.get_messages(thread_id=thread_id)
    print(f"Messages: {messages}")
    
    last_msg = messages.get_last_text_message_by_sender("assistant")
    if last_msg:
        print(f"Last Message: {last_msg.text.value}")
    
    for file_path_annotation in messages.file_path_annotations:
        file_name = os.path.basename(file_path_annotation.text)
        client.agents.save_file(
            file_id=file_path_annotation.file_path.file_id, 
            file_name=file_name, 
            target_dir="./blog"
        )
        print(f"File saved: {file_name}")

In [29]:
def main():
    # Initialize Azure AI Project Client
    project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.getenv("AZURE_PROJ_CONN"),
)

    # Create agent
    agent = create_code_interpreter_agent(project_client)

    # Take user input from CLI
    user_input = input("Enter your request for the agent: ")

    # Send message
    thread = create_thread_and_message(project_client, agent.id, user_input)
    
    # Execute agent logic
    run = execute_run(project_client, thread.id, agent.id)

    # Fetch results
    if run:
        fetch_and_save_file(project_client, thread.id)

if __name__ == "__main__":
    main()

Run finished with status: RunStatus.COMPLETED
Messages: <azure.ai.projects.models._patch.ThreadMessages object at 0x0000021FCDEE6F60>
Last Message: The blog post has been saved successfully. You can download it using the link below:

[Download blog-2025-02-15_13-12-06.md](sandbox:/mnt/data/blog-2025-02-15_13-12-06.md)
File saved: blog-2025-02-15_13-12-06.md
